In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import random

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


W, H, D = 224, 224, 3
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

# model = tf.keras.models.load_model('resnet50.h5')
model = tf.keras.applications.resnet50.ResNet50(
    include_top=0,
    weights="imagenet",
#     input_tensor=None,
    input_shape=(W,H,D),
    pooling='avg',
)

df = pd.read_csv('whale_encodings.csv')
encodings = [' '.join(enc.split()) for enc in list(df['encodings'])]
for i, arr in enumerate(encodings):
    encodings[i] = arr.replace('[', '')
    encodings[i] = encodings[i].replace(']', '')
    
for i, arr in enumerate(encodings):
    encodings[i] = arr.split()[:]
    for j in range(256):
        encodings[i][j] = float(encodings[i][j])

In [ ]:
path = r'F:\hac\custom_dataset'
names = os.listdir(path)
score = [0, 0, 0, 0, 0] # top5_acc
data = pd.DataFrame()
df2 = df.copy()
df2['ids'] = [s.split('\\')[0] for s in list(df['ids'])]

for k in range(len(names)):
#     k += 34
    if k >= len(names):
        break
    print('#'*20,names[k],'#'*20)
    data['ids'] = list(df2['ids'])
    img_names = os.listdir(os.path.join(path, names[k]))
    img_names = random.sample(img_names, 10)
    for i, name in enumerate(img_names):
        img = image.load_img(os.path.join(path, names[k], name), target_size=(W,H,D))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis=0)
        img_tensor /= 255.
        pred = model.predict(img_tensor)
        dist = []
        for j in range(len(encodings_)):
                dist.append(euclidean_distance([pred, encodings_[j]]).numpy()[0][0])
        
        data['dist'] = dist
        mean_dist = []
        for l in list(df2['ids'].value_counts().keys()):
            mean_dist.append(data[data['ids']==l].mean().dist)
        new = pd.DataFrame()
        new['ids'] = list(df2['ids'].value_counts().keys())
        new['mean'] = mean_dist
        
        new = new.sort_values(by='mean')
            
        classes = []
        count = 0
        while len(classes) != len(score):
            cls = new['ids'].iloc[count]
            count += 1
            if not cls in classes:
                classes.append(cls)
        
        for j in range(len(classes)):
            if classes[j] == names[k]:
                score[j] += 1
                break
        print(classes, end=' ')
        print(score)
